In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
from glob import glob

import matplotlib.pyplot as plt

import rioxarray as rio
from shapely.geometry import mapping

%matplotlib inline
# %matplotlib widget

In [2]:
# wrap locating of the median points into a function: 

### (1) read .shp

In [3]:
hma = gpd.read_file('HMA_regions/HMA_regions.shp')
# hma.plot()

# as dataframe
hma_df = pd.DataFrame(hma) 

### (2) read climate

In [4]:
climate = xr.open_mfdataset('*.nc', decode_coords="all")
# cnange units (and overwrite the metadata with the units after)

# ------ UNITS ------

#convert temperature K to C
climate['t2m']=climate.t2m-273.15
# precipotation m to mm 
climate['tp']=climate.tp * 1000
# radiation j/m2 to w/m2
# SSR [W/m2] = SSR [J/m^2] / (3600 seconds)
climate['ssrd'] = climate.ssrd / 3600

# ------ CRS ----------

# add crs to the dataset (netcdf file) - maybe this is not even needed 
climate = climate.rio.write_crs('EPSG:4326')



In [5]:
#

climate_median = climate.median(dim='time', keep_attrs = True, skipna=True)

# climate_median

In [6]:
# hma = hma[hma['Name'] == hma_df.Name[21]]

hma_df = hma_df[:3]

In [13]:
# hma_df

### (3) function: 

In [7]:
# function for median: 

def median_ceil(values):
    values = sorted(values)
    values_len = len(values)
    middle = values_len//2
    return values[middle]


In [7]:
# climate_median.t2m.plot()

In [10]:
hma.geometry.values
# hma_df.values

<GeometryArray>
Length: 22, dtype: geometry

In [12]:
%%time
# for every geometry in hma

# t2m_median = roi_climate_median_df[roi_climate_median_df.t2m==median_ceil(values=roi_climate_median_df.t2m)]

name = []
value = []
lats = []
lons = []


for r in hma.geometry:
        
    # name:
    # roi_name = str(hma.Name.values[0])
    # name = np.append(name, roi_name)
    # roi_ind = r.index    
    # print(roi_ind)
    # name = np.append(name, roi_ind)
    
    # clip:
    clipped = climate_median.rio.clip(r.geometry, r.crs, drop=True, invert=False)
    
    # sanity check
    # clipped.t2m.plot()
    
    # to dataframe:
    df = clipped.to_dataframe().reset_index()
    df = df.dropna().drop_duplicates()
    
    # median vallues for a parameter: 
    median_value = median_ceil(values=df.t2m)
    # print('temp median value', median_value)
    
    # find index of this value:
    row = df[df.t2m == median_value]
    lat = row.latitude.values[0]
    lon = row.longitude.values[0]
    
    print(lat)
    
    #add:
    value = np.append(value, median_value)
    lats = np.append(lats, lat)
    lons = np.append(lons, lon)

    # print('name:', name)
    # print('lat', lat)
    # print('lon', lon)

    

AttributeError: 'Polygon' object has no attribute 'geometry'

lats
lons
value
name

In [ ]:
table = pd.DataFrame([name, lats, lons, value]).transpose()

In [ ]:
table

In [ ]:
name

In [23]:
lons

array([90.5, 90.5, 90.5])